<a href="https://colab.research.google.com/github/yohanesnuwara/MLGeo/blob/main/KAN/KAN_Lithology_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/yohanesnuwara/MLGeo

Cloning into 'MLGeo'...
remote: Enumerating objects: 222, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 222 (delta 43), reused 29 (delta 29), pack-reused 163
Receiving objects: 100% (222/222), 45.17 MiB | 10.42 MiB/s, done.
Resolving deltas: 100% (102/102), done.
Updating files: 100% (39/39), done.


In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable
from pandas import set_option

set_option("display.max_rows", 10)
pd.options.mode.chained_assignment = None

In [3]:
# read data into python as dataframe
dataset = '/content/MLGeo/KAN/feature_vectors_preprocessed.csv'
training_data = pd.read_csv(dataset)
training_data

,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS,Facies
0,77.450,0.664,9.900,11.915,4.600,1,1.000,3
1,78.260,0.661,14.200,12.565,4.100,1,0.979,3
2,79.050,0.658,14.800,13.050,3.600,1,0.957,3
3,86.100,0.655,13.900,13.115,3.500,1,0.936,3
4,74.580,0.647,13.500,13.300,3.400,1,0.915,3
...,...,...,...,...,...,...,...,...
2812,46.719,0.947,1.828,7.254,3.617,2,0.685,5
2813,44.563,0.953,2.241,8.013,3.344,2,0.677,5
2814,49.719,0.964,2.925,8.013,3.190,2,0.669,5
2815,51.469,0.965,3.083,7.708,3.152,2,0.661,5


In [4]:
# apply standard scaling
from sklearn import preprocessing

feature_vectors = training_data.iloc[:,:-1]
correct_facies_labels = training_data.iloc[:,-1]

scaler = preprocessing.StandardScaler().fit(feature_vectors)
scaled_features = scaler.transform(feature_vectors)

scaled_features

array([[ 0.38266965,  0.05650048,  1.13742537, ...,  0.99709923,
        -0.98835323,  1.68780719],
       [ 0.40918233,  0.04427786,  1.91652565, ...,  0.43979265,
        -0.98835323,  1.61456734],
       [ 0.43504038,  0.03205525,  2.02523731, ..., -0.11751394,
        -0.98835323,  1.53783989],
       ...,
       [-0.52501328,  1.27876184, -0.12634776, ..., -0.57450534,
         1.01178402,  0.53340773],
       [-0.4677328 ,  1.28283604, -0.09772035, ..., -0.61686064,
         1.01178402,  0.50550684],
       [-0.51480099,  1.30320706, -0.18360257, ..., -0.45747096,
         1.01178402,  0.47760594]])

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
        scaled_features, correct_facies_labels, test_size=0.2, random_state=10)

## Building Kolmogorov-Arnolds Network

In [6]:
# Install imodelsx to use KAN
!pip install imodelsx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.2/237.2 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 23.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302

In [7]:
from imodelsx import KANClassifier
from sklearn.metrics import f1_score


Training with KAN

In [8]:
# Build default KAN model. Use cuda as device if using GPU
# Default hyperparams: hidden_layer_size=64, regularize_activation=1, regularize_entropy=1, regularize_rige=0
model = KANClassifier(device='cuda')

# Fit with training data
model.fit(X_train, y_train)

# Evaluate F1 accuracy on train set
y_train_pred = model.predict(X_train)
f1_train = f1_score(y_train, y_train_pred, average='weighted')
print(f1_train)

# Evaluate F1 accuracy on test set
y_test_pred = model.predict(X_test)
f1_test = f1_score(y_test, y_test_pred, average='weighted')
print(f1_test)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 100/100 [00:06<00:00, 14.78it/s]


0.29527345027559665
0.31644557716833355


F1 score Not good !

So try hyperparameter grid search for the hyperparameters.

Result - hidden layer size around 256, activation around 0.5, entropy around 0.5, ridge around 0.1 --> F1 score train 0.461, F1 score test 0.469

50.8% improvement in F1 score testing, but the score is still Low, not acceptable

In [ ]:
# Hyperparameter grid
layers = [32, 64, 128, 256]
activation = [0.1, 0.5, 0.9]
entropy = [0.1, 0.5, 0.9]
ridge = [0.1, 0.5, 0.9]

# varying hyperparameter
for hidden_layer_size in layers:
  for regularize_activation in activation:
    for regularize_entropy in entropy:
      for regularize_ridge in ridge:
        model = KANClassifier(hidden_layer_size=hidden_layer_size, device='cuda',
                                      regularize_activation=regularize_activation,
                                      regularize_entropy=regularize_entropy,
                                       regularize_ridge=regularize_ridge)

        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        f1_train = f1_score(y_train, y_train_pred, average='weighted')

        y_test_pred = model.predict(X_test)
        f1_test = f1_score(y_test, y_test_pred, average='weighted')

        print(hidden_layer_size, regularize_activation, regularize_entropy, regularize_ridge, f1_train, f1_test)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 100/100 [00:07<00:00, 13.58it/s]


32 0.1 0.1 0.1 0.22511235206857888 0.2108699017652348


100%|██████████| 100/100 [00:04<00:00, 20.41it/s]


32 0.1 0.1 0.5 0.17190381921867456 0.1298139912681913


100%|██████████| 100/100 [00:03<00:00, 26.23it/s]


...

Early stopping
256 0.9 0.9 0.1 0.40869991856631066 0.41522993904123867


 61%|██████    | 61/100 [00:02<00:01, 24.79it/s]


Early stopping
256 0.9 0.9 0.5 0.4340779533098317 0.4499503556737508


100%|██████████| 100/100 [00:04<00:00, 21.81it/s]

256 0.9 0.9 0.9 0.44061205047076696 0.45416716097476967


Try more refined hyperparameter grid search to hit the best value

We still get improvement using hidden layer size 1024, activation 0.5, entropy 0.4, and ridge 0.2 --> F1 score training 0.48, F1 score testing 0.50

Another 60.7% improvement compared to first one! But we can improve further

In [ ]:
# Hyperparameter grid
layers = [256, 512, 1024]
activation = [0.4, 0.5, 0.6]
entropy = [0.4, 0.5, 0.6]
ridge = [0.05, 0.1, 0.2]

# varying hyperparamater
for hidden_layer_size in layers:
  for regularize_activation in activation:
    for regularize_entropy in entropy:
      for regularize_ridge in ridge:
        model = KANClassifier(hidden_layer_size=hidden_layer_size, device='cuda',
                                      regularize_activation=regularize_activation,
                                      regularize_entropy=regularize_entropy,
                                       regularize_ridge=regularize_ridge)

        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        f1_train = f1_score(y_train, y_train_pred, average='weighted')

        y_test_pred = model.predict(X_test)
        f1_test = f1_score(y_test, y_test_pred, average='weighted')

        print(hidden_layer_size, regularize_activation, regularize_entropy, regularize_ridge, f1_train, f1_test)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 100/100 [00:04<00:00, 20.34it/s]


256 0.4 0.4 0.05 0.37847402993236184 0.37030663954894316


100%|██████████| 100/100 [00:04<00:00, 21.88it/s]


256 0.4 0.4 0.1 0.40786878859811465 0.4021884686558395


100%|██████████| 100/100 [00:03<00:00, 25.21it/s]


...

1024 0.6 0.6 0.05 0.48158948690977177 0.4890730567716075


100%|██████████| 100/100 [00:06<00:00, 14.50it/s]


1024 0.6 0.6 0.1 0.47923311280483366 0.4885412728414198


100%|██████████| 100/100 [00:07<00:00, 12.96it/s]

1024 0.6 0.6 0.2 0.4784652359969078 0.4835748313408062


Let's use these hyperparams

In [9]:
model = KANClassifier(hidden_layer_size=1024, regularize_activation=.5,
                      regularize_entropy=.4, regularize_ridge=.2,
                      device='cuda')

# Fit with training data
model.fit(X_train, y_train)

# Evaluate F1 accuracy on train set
y_train_pred = model.predict(X_train)
f1_train = f1_score(y_train, y_train_pred, average='weighted')
print(f1_train)

# Evaluate F1 accuracy on test set
y_test_pred = model.predict(X_test)
f1_test = f1_score(y_test, y_test_pred, average='weighted')
print(f1_test)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 100/100 [00:12<00:00,  7.70it/s]

0.4781286831240907
0.496057076634108


We have another configuration for training. We can do another grid search to find the best configuration.

We get improvement to F1 score training 0.57 and F1 score testing 0.58.

In [15]:
# Grid of training configuration
batch = [64, 128, 512]
learning = [0.001, 0.005, 0.01, 0.05]
weights = [0.01, 0.05, 0.1]
gammas = [0.1, 0.5, 0.8]

# Use tuned hyperparameter
model = KANClassifier(hidden_layer_size=1024, regularize_activation=.5,
                      regularize_entropy=.4, regularize_ridge=.2,
                      device='cuda')

# Varying training configuration
for batch_size in batch:
  for lr in learning:
    for weight_decay in weights:
      for gamma in gammas:
        model.fit(X_train, y_train, batch_size=batch_size, lr=lr,
                  weight_decay=weight_decay, gamma=gamma)

        y_train_pred = model.predict(X_train)
        f1_train = f1_score(y_train, y_train_pred, average='weighted')

        y_test_pred = model.predict(X_test)
        f1_test = f1_score(y_test, y_test_pred, average='weighted')

        print(batch_size, lr, weight_decay, gamma, f1_train, f1_test)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 100/100 [00:19<00:00,  5.09it/s]


64 0.001 0.01 0.1 0.014894725960753016 0.015565994289398541


 10%|█         | 10/100 [00:01<00:17,  5.08it/s]


Early stopping
64 0.001 0.01 0.5 0.5013093690822 0.5457395104988801


  4%|▍         | 4/100 [00:00<00:21,  4.48it/s]


...

512 0.05 0.1 0.1 0.5022245814212029 0.5457145999558024


 26%|██▌       | 26/100 [00:02<00:06, 12.11it/s]


Early stopping
512 0.05 0.1 0.5 0.5506310632332884 0.545749793864033


  7%|▋         | 7/100 [00:00<00:07, 12.86it/s]

Early stopping
512 0.05 0.1 0.8 0.5523397290227592 0.5263521910484665


Further refine the grid search.

Finally we have F1 score training 0.58 and F1 score testing 0.6,

Using configuration as follows: batch size 64, learning rate 0.07, weight decay 0.03, and gamma 0.6

Neat !!

In [17]:
# Grid of training configuration
batch = [64, 128, 512]
learning = [0.05, 0.07, 0.1]
weights = [0.005, 0.01, 0.03]
gammas = [0.4, 0.5, 0.6]

# Use tuned hyperparameter
model = KANClassifier(hidden_layer_size=1024, regularize_activation=.5,
                      regularize_entropy=.4, regularize_ridge=.2,
                      device='cuda')

# Varying training configuration
for batch_size in batch:
  for lr in learning:
    for weight_decay in weights:
      for gamma in gammas:
        model.fit(X_train, y_train, batch_size=batch_size, lr=lr,
                  weight_decay=weight_decay, gamma=gamma)

        y_train_pred = model.predict(X_train)
        f1_train = f1_score(y_train, y_train_pred, average='weighted')

        y_test_pred = model.predict(X_test)
        f1_test = f1_score(y_test, y_test_pred, average='weighted')

        print(batch_size, lr, weight_decay, gamma, f1_train, f1_test)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
  3%|▎         | 3/100 [00:01<00:44,  2.19it/s]


Early stopping
64 0.05 0.005 0.4 0.5687751716293812 0.5821815867541443


  9%|▉         | 9/100 [00:03<00:36,  2.49it/s]


Early stopping
64 0.05 0.005 0.5 0.5721988024828358 0.5653255285134674


  3%|▎         | 3/100 [00:00<00:30,  3.17it/s]


...

Early stopping
512 0.1 0.03 0.4 0.5415586789601221 0.5317056786349507


 25%|██▌       | 25/100 [00:02<00:06, 12.29it/s]


Early stopping
512 0.1 0.03 0.5 0.5413927468832567 0.5660821177311369


 39%|███▉      | 39/100 [00:03<00:04, 12.61it/s]


Early stopping
512 0.1 0.03 0.6 0.5528386774205456 0.5521016539316563


In [32]:
### Final KAN model
model = KANClassifier(hidden_layer_size=1024, regularize_activation=.5,
                      regularize_entropy=.4, regularize_ridge=.2,
                      device='cuda')

# Fit with training data
model.fit(X_train, y_train, batch_size=64, lr=.07,
                  weight_decay=.03, gamma=.5)

# Evaluate F1 accuracy on train set
y_train_pred = model.predict(X_train)
f1_train = f1_score(y_train, y_train_pred, average='weighted')
print(f1_train)

# Evaluate F1 accuracy on test set
y_test_pred = model.predict(X_test)
f1_test = f1_score(y_test, y_test_pred, average='weighted')
print(f1_test)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
  4%|▍         | 4/100 [00:00<00:21,  4.41it/s]


Early stopping
0.5857933762051516
0.5958239601775241


In [37]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           1       0.61      0.66      0.64       203
           2       0.60      0.57      0.59       506
           3       0.67      0.68      0.67       442
           4       0.56      0.66      0.61       119
           5       0.35      0.19      0.25       142
           6       0.54      0.58      0.56       332
           7       0.55      0.66      0.60        65
           8       0.52      0.56      0.54       306
           9       0.81      0.66      0.73       138

    accuracy                           0.59      2253
   macro avg       0.58      0.58      0.58      2253
weighted avg       0.59      0.59      0.59      2253

              precision    recall  f1-score   support

           1       0.68      0.75      0.72        52
           2       0.62      0.62      0.62       114
           3       0.66      0.59      0.62       105
           4       0.57      0.54      0.56        37
           5       0.21 

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
